### yes24 Crawling / 예스24 크롤링

In [1]:
# webdriver_manager 라이브러리 불러오기
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()

'/Users/jeonmh/.wdm/drivers/chromedriver/mac64/120.0.6099.109/chromedriver-mac-arm64/chromedriver'

In [7]:
from selenium import webdriver
browser = webdriver.Chrome()

In [14]:
url = 'https://google.com'
browser.get(url)

In [15]:
from selenium.webdriver.common.by import By
browser.find_element(By.CLASS_NAME, 'uU7dJb').text

'대한민국'

In [16]:
url = 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=3&pageSize=24'
browser.get(url)


In [17]:
# 1페이지의 링크 데이터 전부 수집
### 한개의 베스트 셀러 링크 수집
browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

### 1페이지의 전체의 링크 데이터 전부 수집
datas = browser.find_elements(By.CLASS_NAME, 'gd_name')
##### element : 요소 / elements : 리스트

for i in datas:
    print(i.get_attribute('href'))

# <a class="gd_name" href="/Product/Goods/122120495" onclick="wiseLogV2('BS', '001_005_001', ''); ">마흔에 읽는 쇼펜하우어</a>
# <a class="gd_name" href="/Product/Goods/117014613" onclick="wiseLogV2('BS', '001_005_001', ''); ">세이노의 가르침</a>


https://www.yes24.com/Product/Goods/90428162
https://www.yes24.com/Product/Goods/123814844
https://www.yes24.com/Product/Goods/123318244
https://www.yes24.com/Product/Goods/122865851
https://www.yes24.com/Product/Goods/122786183
https://www.yes24.com/Product/Goods/122433629
https://www.yes24.com/Product/Goods/124225344
https://www.yes24.com/Product/Goods/124315515
https://www.yes24.com/Product/Goods/9349031
https://www.yes24.com/Product/Goods/124472702
https://www.yes24.com/Product/Goods/121911671
https://www.yes24.com/Product/Goods/122880691
https://www.yes24.com/Product/Goods/8759796
https://www.yes24.com/Product/Goods/123153430
https://www.yes24.com/Product/Goods/124106975
https://www.yes24.com/Product/Goods/16384156
https://www.yes24.com/Product/Goods/96794060
https://www.yes24.com/Product/Goods/69655504
https://www.yes24.com/Product/Goods/116020437
https://www.yes24.com/Product/Goods/106369008
https://www.yes24.com/Product/Goods/124411457
https://www.yes24.com/Product/Goods/106711

In [62]:
# 3페이지까지의 링크 데이터 전부 수집
import time

link_list = []
for i in range(1, 4):
    print('*'*3,f'현재 {i} 페이지 수집 중 입니다.','*'*3)
    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'

    browser.get(url)
    datas = browser.find_elements(By.CLASS_NAME, 'gd_name')
    
    for a in datas:
        link = a.get_attribute('href')
        link_list.append(link)

    time.sleep(3)
    

*** 현재 1 페이지 수집 중 입니다. ***
*** 현재 2 페이지 수집 중 입니다. ***
*** 현재 3 페이지 수집 중 입니다. ***


In [19]:
len(link_list)

72

In [46]:
# # 상세 페이지로 이동
# # for link in link_list:
# #     browser.get(link)

# browser.get(link_list[0])

# # 제목
# title = browser.find_element(By.CLASS_NAME, 'gd_name').text

# # 저자
# auth = browser.find_element(By.CLASS_NAME, 'gd_auth').text

# # 출판사
# publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text

# # 출간일자 2024-01-01 식으로 변경
# publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text

# # 평점
# reting = browser.find_element(By.CLASS_NAME, 'yes_b').text

# # 리뷰
# review = browser.find_element(By.CLASS_NAME, 'txC_blue').text

# # 판매지수
# sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(' ')[2]

# # 판매가
# price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]

# # 국내랭킹
# ranking = browser.find_element(By.CLASS_NAME, 'gd_best ,gd_best_tp02').find_element(By.TAG_NAME, 'a').text

# # 국내랭킹 진척도
# if browser.find_element(By.CLASS_NAME, 'gd_best,gd_best_tp02').text.count('|') == 1:
#     ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best,gd_best_tp02').find_element(By.TAG_NAME, 'dd').text.split(' | ')[1]
# else:
#     pass


'국내도서 top20 3주'

In [61]:
# 데이터 베이스 연동 후 -> 수집한 데이터를 DB에 저정 (CSV)
import pymysql
import time
import re
from datetime import datetime

conn = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = '4887',
    db = 'yes24s',
    charset = 'utf8mb4',
    cursorclass = pymysql.cursors.DictCursor
)

with conn.cursor() as cur:

    for i in link_list:
        browser.get(i)

        # 상세 페이지 이동 후 데이터 크롤링
        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
        if match:
            year,month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day))
            publishing = data_obj.strftime('%Y-%m-%d')
        else:
            publishing = '2024-01-01'
        if rating:
            reting = browser.find_element(By.CLASS_NAME, 'yes_b').text
            review= browser.find_element(By.CLASS_NAME, 'txC_blue').text
            review = int(review.replace(",",""))
        else:
            rating = None
            review = None
        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(' ')[2]
        sales = int(sales.replace(",",""))
        price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
        price = int(price.replace(",",""))
        ranking = browser.find_element(By.CLASS_NAME, 'gd_best ,gd_best_tp02').find_element(By.TAG_NAME, 'a').text.split(' ')[1][:-1]
        if browser.find_element(By.CLASS_NAME, 'gd_best,gd_best_tp02').text.count('|') == 1:
            ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best,gd_best_tp02').find_element(By.TAG_NAME, 'dd').text.split(' | ')[1][-3:-1]
        else:
            ranking_weeks = None


        sql = """INSERT INTO Books(
            title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks
            )
            VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )
            """

        cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
        conn.commit()

        time.sleep(3)

In [ ]:
import pymysql
import time
import re
from datetime import datetime

conn = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = '4887',
    db = 'yes24s',
    charset = 'utf8mb4',
    cursorclass = pymysql.cursors.DictCursor
)

with conn.cursor() as cur:
    for i in link_list:
        browser.get(i)
        
        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        if browser.find_element(By.CLASS_NAME, 'yes_b').text :
            rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
            review = browser.find_element(By.CLASS_NAME, 'gd_reviewCount').find_element(By.TAG_NAME, 'em').text
            review = int(review.replace(",",""))
        
        sql = """UPDATE Books SET rating = %s, review = %s WHERE title = %s"""

        cur.execute(sql, (rating, review,title))
        conn.commit()
        time.sleep(3)

In [101]:
# 오류 확인용 코드들
import re
# 상세 페이지 이동 후 데이터 크롤링
browser.get(link_list[38])

title = browser.find_element(By.CLASS_NAME, 'gd_name').text
auth = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
if match:
    year,month, day = match.groups()
    data_obj = datetime(int(year), int(month), int(day))
    publishing = data_obj.strftime('%Y-%m-%d')
else:
    publishing = '2024-01-01'
if browser.find_element(By.CLASS_NAME, 'yes_b').text :
    rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
    review = browser.find_element(By.CLASS_NAME, 'gd_reviewCount').find_element(By.TAG_NAME, 'em').text
    review = int(review.replace(",",""))
    # if browser.find_element(By.CLASS_NAME, 'gd_reviewCount').text.split(' ')[-2].isdigit  == True:
    #     review = browser.find_element(By.CLASS_NAME, 'gd_reviewCount').text.split(' ')[-1]
    # else:
    #     review = None
else:
    rating = None
    review = None

sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(' ')[2]
sales = int(sales.replace(",",""))
price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
price = int(price.replace(",",""))
ranking = browser.find_element(By.CLASS_NAME, 'gd_best ,gd_best_tp02').find_element(By.TAG_NAME, 'a').text.split(' ')[1][:-1]
if browser.find_element(By.CLASS_NAME, 'gd_best,gd_best_tp02').text.count('|') == 1:
    ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best,gd_best_tp02').find_element(By.TAG_NAME, 'dd').text.split(' | ')[1][-3:-1]
else:
    ranking_weeks = None

title, auth, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks
# ranking

('흔한남매 과학 탐험대 0 과학의 기초',
 '흔한남매 원저/김언정, 이현진 글/김덕영 그림/최진수 감수 외 4명 정보 더 보기/감추기',
 '주니어김영사',
 '2023-12-27',
 '9.7',
 27,
 86709,
 14800,
 '39',
 None)